# M5のおさらい

- M5ではtreeの形成は普通のdecision treeと同じように行う（？）
    - これは作りたいモデルとはたぶん異なる。
    

# M5PP

- ふたつに分割するアルゴリズムを作る
    - これをrecursiveに適用すればcascading treeになるはず
- 分割にあたっては線形モデルのエラーを最小化するようにする
- ただし、ifelse型のスイッチは最適化困難なので、sigmoidかtanhを用いる。
- 以下のようなmodelを小さい$a$でまずtrainし、徐々に$a$を大きくしていくのがいいだろうか？
　　$$\left[\tanh\left(a(x+b)\right) -1\right]\times\text{回帰式}$$
  
- regime switch用のファクターごとにモデルを作って比較するのが安直ではあるが、もっとスマートな方法はあるか

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\t\Anaconda3\envs\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
Nsample = 1000

In [6]:
Nfactor = 10

In [7]:
trX = np.random.randn(Nsample , Nfactor)

In [9]:
trX.shape

(1000, 10)

In [11]:
Nregime  = 2

In [13]:
A = np.random.randn(Nfactor , Nregime)

In [2]:
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33 # create a y value which is approximately linear but with some random noise

In [17]:
trZ = np.dot(trX , A)

In [18]:
trZ.shape

(1000, 2)

In [19]:
trZ[1].shape

(2,)

In [29]:
Nregime_factor = 2

In [30]:
trX2 = np.random.randn(Nsample, Nregime_factor)

In [31]:
A2 = np.random.randn(Nregime_factor , 1)

In [33]:
trZ2 = np.dot(trX2 , A2)

In [21]:
trX2.shape

(1000, 1)

M5やM5Pに倣い、regime switcherとしては、候補とするファクターひとつのみが含まれる不等式をカスケード式に用いることを想定

従って


In [34]:
trY = [z[0] + z[1] if  trZ2[i , 0] > 0 else z[0] - z[1] for i , z in enumerate(trZ)]

In [35]:
def model(X, X2 , w, w2): # this network is the same as the previous one except with an extra hidden layer + dropout
    z = tf.matmul(X , w)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(z2)
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))

    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))

    h2 = tf.nn.dropout(h2, p_keep_hidden)


    return tf.matmul(h2, w_o)